In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
%load_ext tensorboard

In [2]:
target_size = (150, 150)
input_shape = (target_size[0], target_size[1], 3)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 5,
    brightness_range = (0.5,1),
    shear_range = 5,
    preprocessing_function=tf.image.rgb_to_grayscale
)

print("Training:\n")
traingen = datagen.flow_from_directory(
    directory = "datasets/asl_alphabet/train",
    target_size = target_size,
    seed = 1337
)

print("Validation:\n")
valgen = datagen.flow_from_directory(
    directory = "datasets/asl_alphabet/val",
    target_size = target_size,
    seed = 1337
)

print("Test:\n")
testgen = datagen.flow_from_directory(
    directory = "datasets/asl_alphabet/test",
    target_size = target_size,
    seed = 1337
)

Training:

Found 69600 images belonging to 29 classes.
Validation:

Found 8700 images belonging to 29 classes.
Test:

Found 8700 images belonging to 29 classes.


In [3]:
tf.keras.backend.clear_session()

cnn = tf.keras.models.Sequential()

cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same" , activation="relu", input_shape=input_shape))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(filters=64, kernel_size=(2, 2), padding="same" , activation="relu", input_shape=input_shape))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())

cnn.add(Dense(units = 256, activation = "relu"))

cnn.add(Dense(units = 29, activation = "softmax"))

In [4]:
cnn.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 87616)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               22429952  
_________________________________________________________________
dense_1 (Dense)              (None, 29)                7

In [5]:
! takeown /r /f logs 1>nul
! cacls logs /c /G "ADMINNAME":F /T 1>nul
! del /f /s /q logs 1>nul
! rmdir /s /q logs 1>nul

No mapping between account names and security IDs was done.



In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
%tensorboard --logdir logs/fit

In [8]:
mcp_save = ModelCheckpoint("simple_cnn.h5", save_best_only=True, monitor="accuracy", mode="max")

history = cnn.fit(
    traingen,
    epochs = 18,
    validation_data = valgen,
    callbacks=[mcp_save, tensorboard_callback]
)

Epoch 1/18
   1/2175 [..............................] - ETA: 0s - loss: 3.3182 - accuracy: 0.0312WARNING:tensorflow:From C:\Users\yanco\.conda\envs\TensorFlow\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2175/2175 [==============================] - 1482s 681ms/step - loss: 1.5098 - accuracy: 0.5455 - val_loss: 0.6325 - val_accuracy: 0.7955
Epoch 2/18
2175/2175 [==============================] - 1408s 648ms/step - loss: 0.4423 - accuracy: 0.8544 - val_loss: 0.3042 - val_accuracy: 0.9003
Epoch 3/18
2175/2175 [==============================] - 1437s 661ms/step - loss: 0.2610 - accuracy: 0.9130 - val_loss: 0.2601 - val_accuracy: 0.9130
Epoch 4/18
2175/2175 [==============================] - 1426s 655ms/step - loss: 0.1823 - accuracy: 0.9399 - val_loss: 0.2190 - val_accuracy: 0.9257
Epoch 5/18
2175/2175 [=